# Initialization

In [1]:
import os
from pathlib import Path
import signal
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces

# BioLQM init
import biolqm

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

This notebook has been executed using the docker image `colomoto/colomoto-docker:2022-05-01`

<Handlers.SIG_DFL: 0>

In [2]:
# compute trapspaces using PyBoolNet directly
def bench_pyboolnet(models, max=1000, timeout=120):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            primes = bnet2primes(model + ".bnet")
            end = time.perf_counter()
            print(f"implicants: {end - start}s")
            tspaces = compute_trap_spaces(primes, "min", max_output=max)
            end = time.perf_counter()
            print(f"{len(tspaces)} trap spaces {end - start}s\n===")
        except TimeOutException as ex:
            print("DNF\n===")
        signal.alarm(0)

In [3]:
# compute trapspaces using trappist
def bench_trappist(models, max=1000, timeout=120):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            biolqm.save(biolqm.load(model + ".bnet"), model + ".pnml")
            end = time.perf_counter()
            print(f"conversion: {end - start}s")
            tspaces = trappist.compute_trap_spaces(model + ".pnml", max_output=max)
            end = time.perf_counter()
            print(f"{len(list(tspaces))} trap spaces {end - start}s\n===")
        except TimeOutException as ex:
            print("DNF\n===")
        signal.alarm(0)

## PyBoolNet repository

Unfortunately all models are very easy

In [4]:
# all models from the PyBoolNet repository https://github.com/hklarner/pyboolnet/tree/master/pyboolnet/repository
models = get_all_names()

# only the slowest ones
# models = ['jaoude_thdiff', 'n12c5', 'remy_tumorigenesis', 'selvaggio_emt']

# save .bnet files locally
for model in models:
    net = get_bnet(model)
    with open(model + ".bnet", "wt") as f:
        print(net, file=f)

In [5]:
bench_pyboolnet(models)

arellano_rootstem
implicants: 0.02032073901500553s
4 trap spaces 0.04268817702541128s
===
calzone_cellfate
implicants: 0.013558565988205373s
27 trap spaces 0.0323178640101105s
===
dahlhaus_neuroplastoma
implicants: 0.02779431495582685s
32 trap spaces 0.04492184100672603s
===
davidich_yeast
implicants: 0.01628880196949467s
12 trap spaces 0.03326889296295121s
===
dinwoodie_life
implicants: 0.011097708949819207s
7 trap spaces 0.026619303971529007s
===
dinwoodie_stomatal
implicants: 0.00844807003159076s
1 trap spaces 0.021808986028190702s
===
faure_cellcycle
implicants: 0.02236471400829032s
2 trap spaces 0.04016565001802519s
===
grieco_mapk
implicants: 0.0227065869839862s
18 trap spaces 0.045427492004819214s
===
irons_yeast
implicants: 0.02908623800612986s
1 trap spaces 0.04884503502398729s
===
jaoude_thdiff
implicants: 1.5588138289749622s
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 1.7339413249865174s
===
klamt_tcr
implicants: 

In [6]:
bench_trappist(models)

arellano_rootstem
conversion: 0.2713543900172226s
4 trap spaces 0.30667494097724557s
===
calzone_cellfate
conversion: 0.038661268015857786s
27 trap spaces 0.07898580702021718s
===
dahlhaus_neuroplastoma
conversion: 0.03730725502828136s
32 trap spaces 0.10562571499031037s
===
davidich_yeast
conversion: 0.035403173009399325s
12 trap spaces 0.08253597898874432s
===
dinwoodie_life
conversion: 0.03324870899086818s
7 trap spaces 0.08425742696272209s
===
dinwoodie_stomatal
conversion: 0.016567468992434442s
1 trap spaces 0.049956837960053235s
===
faure_cellcycle
conversion: 0.015124688041396439s
2 trap spaces 0.04512780299410224s
===
grieco_mapk
conversion: 0.036839039996266365s
18 trap spaces 0.07797946402570233s
===
irons_yeast
conversion: 0.012983702996280044s
1 trap spaces 0.0428511860081926s
===
jaoude_thdiff
conversion: 0.09780118701746687s
1000 trap spaces 0.4014113519806415s
===
klamt_tcr
conversion: 0.009428084013052285s
8 trap spaces 0.03665095701580867s
===
krumsiek_myeloid
conversi

## Local set of big models

Collected from various bibliographic sources (see article for details)

In [7]:
# all models from the trappist's benchmark directory
models = [str(Path().absolute() / "models" / name[:-5]) for name in os.listdir("models") if name.endswith(".bnet")]

In [8]:
bench_pyboolnet(models)

/notebook/models/CASCADE3
implicants: 118.48173537303228s
1 trap spaces 118.5426375970128s
===
/notebook/models/KYNURENINE-PATHWAY
implicants: 102.39985360001447s
1 trap spaces 102.45308068604209s
===
/notebook/models/SN5
Timeout
DNF
===
/notebook/models/ERBB-RECEPTOR-SIGNALING
Timeout
DNF
===
/notebook/models/Executable_file_for_cholocystokinin_model
implicants: 0.6582137540099211s
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 1.4686117930104956s
===
/notebook/models/korkut_2015a
Timeout
DNF
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
implicants: 0.17207431397400796s
1 trap spaces 0.24107225897023454s
===
/notebook/models/Corral_ThIL17diff_15jan2021
Timeout
DNF
===
/notebook/models/HIV-1
Timeout
DNF
===
/notebook/models/Colitis_associated_colon_cancer
implicants: 0.10503325203899294s
5 trap spaces 0.22000449802726507s
===
/notebook/models/INTERFERON-1
implicants: 13.10918293800205s
2 trap spaces 13.212684640

In [9]:
bench_trappist(models)

/notebook/models/CASCADE3
conversion: 5.683731729979627s
1 trap spaces 15.423345213988796s
===
/notebook/models/KYNURENINE-PATHWAY
conversion: 1.2255762260174379s
1 trap spaces 5.549958793038968s
===
/notebook/models/SN5
conversion: 88.45769342401763s
Timeout
DNF
===
/notebook/models/ERBB-RECEPTOR-SIGNALING
conversion: 2.166969265032094s
1000 trap spaces 13.773235631990246s
===
/notebook/models/Executable_file_for_cholocystokinin_model
conversion: 0.9507876700372435s
1000 trap spaces 5.097343516012188s
===
/notebook/models/korkut_2015a
conversion: 0.5923634140053764s
1000 trap spaces 3.4736944909673184s
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
conversion: 0.06179241999052465s
1 trap spaces 0.3519580000429414s
===
/notebook/models/Corral_ThIL17diff_15jan2021
conversion: 0.1589940899866633s
1000 trap spaces 1.8983286279835738s
===
/notebook/models/HIV-1
conversion: 0.47149456199258566s
6 trap spaces 2.3135663889697753s
===
/notebook/models/Colitis_associated_colon_ca

Py4JError: An error occurred while calling o0.save